# Set correct upload date and user for subjects and experiments

1. Create all users on the destination
2. Perform the project migration
3. Write subject and experiment metadata to CSV
4. Query the destination database to update the metadata with original upload date and user


In [2]:
import pandas as pd
import xnat

In [3]:
source = xnat.connect("https://ucl-test-xnat.cs.ucl.ac.uk")
destination = xnat.connect("http://localhost", user="admin", password="admin")

[WARNING] Encountered errors retrieving display fields for: sets:definition, cx:clinicalAssessmentData, ml:trainSession, ml:model, sets:collection, ml:trainConfig


## 1. Create users on the destination

In [3]:
source_profiles = source.get("/xapi/users/profiles", format="json").json()
destination_profiles = destination.get("/xapi/users/profiles", format="json").json()

In [4]:
# First check that existing users on the destination are identical to the source
for source_profile, destination_profile in zip(source_profiles, destination_profiles, strict=False):
    if source_profile["username"] != destination_profile["username"]:
        msg = f"Usernames not equal: {source_profile['username']=} {destination_profile['username']=}"
        raise (ValueError(msg))

    if source_profile["id"] != destination_profile["id"]:
        msg = f"IDs not equal: {source_profile['id']=} {destination_profile['id']=}"
        raise (ValueError(msg))

In [5]:
for source_profile in source_profiles[len(destination_profiles) :]:
    print(f"Creating user: {source_profile['username']}")
    destination_profile = {
        "username": source_profile["username"].rstrip("#EXT#"),  # TODO: how to handle external auth?
        "enabled": source_profile["enabled"],
        "email": source_profile["email"],
        "verified": source_profile["verified"],
        "firstName": source_profile["firstName"],
        "lastName": source_profile["lastName"],
    }
    destination.post("/xapi/users", json=destination_profile)

Creating user: mirsg_service
Creating user: rmhihch
Creating user: rmapajw
Creating user: rmapjp0
Creating user: HMTest01
Creating user: HitMSync
Creating user: RTTQA_U1
Creating user: HMTest_Site_1_mgr
Creating user: ctc_xnat_manager
Creating user: cceadm3
Creating user: CaptProton
Creating user: festival_demo
Creating user: site-uploader
Creating user: project-coordinator
Creating user: rmaprmg
Creating user: ccaepsm
Creating user: mailing_list_test
Creating user: hrc_standard_user
Creating user: ucaspjr
Creating user: haroonrchughtai_gmail.com#EXT#


## 2. Perform the project migration

This is the existing functionality of in `xmigrate.main.Migration.run`

## 3. Write subject and experiment metadata to CSV

In [23]:
project_id = "festival_demo"
params = {"columns": "ID,label,insert_user,insert_date,last_modified", "format": "json"}

# Subject metadata
subject_metadata = source.get(f"/data/projects/{project_id}/subjects", query=params)
df = pd.DataFrame(subject_metadata.json()["ResultSet"]["Result"])
df.to_csv("subject_metadata.csv", index=False)

# Experiment metadata
experiment_metadata = source.get(f"/data/projects/{project_id}/experiments", query=params)
df = pd.DataFrame(experiment_metadata.json()["ResultSet"]["Result"])
df.to_csv("experiment_metadata.csv", index=False)

## 4. Query the destination database to update the metadata with original upload date and user

### Check existing subject metadata on the destination:
```sql
SELECT 
    s.project,
    s.label,
    s.id,
    m.insert_date,
    m.last_modified,
    u.login AS insert_user,
    m.insert_user_xdat_user_id AS insert_user_id
FROM 
    xnat_subjectdata s
JOIN 
    xnat_subjectdata_meta_data m ON s.subjectdata_info = m.meta_data_id
LEFT JOIN 
    xdat_user u ON m.insert_user_xdat_user_id = u.xdat_user_id
WHERE 
    s.project = '<PROJECT_ID>';
```

### Update existing subject metadata on the destination:
```sql
-- source_subjects is the metadata from the source xnat
UPDATE xnat_subjectdata_meta_data m
SET 
    insert_date = source.insert_date,
    last_modified = source.last_modified,
    insert_user_xdat_user_id = source.insert_user_xdat_user_id  -- the integer ID, not username
FROM 
    xnat_subjectdata s
JOIN 
    source_subjects source ON s.label = source.label
WHERE 
    s.subjectdata_info = m.meta_data_id
    AND s.project = source.project;  -- project ID
```

### Check existing experiment metadata on the destination:
```sql
SELECT 
    e.project,
    e.label,
    e.id,
    m.insert_date,
    m.last_modified,
    u.login AS insert_user,
    m.insert_user_xdat_user_id AS insert_user_id
FROM 
    xnat_experimentdata e
JOIN 
    xnat_experimentdata_meta_data m ON e.experimentdata_info = m.meta_data_id
LEFT JOIN 
    xdat_user u ON m.insert_user_xdat_user_id = u.xdat_user_id
WHERE 
    e.project = '<PROJECT_ID>';
```

### Update existing experiment metadata on the destination:
```sql
-- source_experiments is the metadata from the source xnat
UPDATE xnat_experimentdata_meta_data m
SET 
    insert_date = source.insert_date,
    last_modified = source.last_modified,
    insert_user_xdat_user_id = source.insert_user_xdat_user_id  -- the integer ID, not username
FROM 
    xnat_experimentdata e
JOIN 
    source_experiments source ON e.label = source.label
WHERE 
    e.experimentdata_info = m.meta_data_id
    AND e.project = source.project;  -- project ID
```
